In [296]:
import pandas as pd
import re
import os
import string
from pprint import pprint
import unicodedata
import pandas as pd 

In [297]:
#Схраняем в переменную directory путь, где лежит наш датасет
directory = "/Users/Anastasia/Documents/Учеба в Вышке_Computer linguistics/First_project/GoldenStandard_raws"

In [474]:
#В file записываем все файлы нашего датасета
files = os.listdir(directory)
#files

['.DS_Store',
 'Pic-RUS_corpus_raw.txt',
 'NDS_corpus_raw.txt',
 'Sib_corpus_raw.txt',
 '.ipynb_checkpoints',
 'the whole_dataset.csv',
 'the whole_dataset.xlsx',
 'preprocessing_GS.ipynb']

In [489]:
#Читаем и обрабатываем каждый файл нашего датасета:
for f in files[1:4]:
    with open (os.path.join(directory, f),'r', encoding ='utf-8') as file:
        text_spld = []
        for i in file:
            text = file.readlines()
            #pprint(text)
            for i in text: #Разбиваем список по табуляции 
                new_text = i.split('\t')
                text_spld.append(new_text)
            for i in text_spld: #Удаляем пустые элементы из списка
                del i[0]
             
    #Работаем с первыми элементами списка - текстом. Убираем лишнюю разметку из текста.
    #Порядок циклов важен!   
    #Убираем удлиненные фонемы: "су-умку" к "сумку", "О-он" --> "Он":  
    #Но исключаем такие случаи (З-з= ‖ заглянул) - здесь нужно убирать больше
    #И не трогаем случаи c предлогом "из-за"
    
    for st in text_spld:
        if len(re.findall(r'(\b[иИ]з-за\b)', st[0])) != 0: #Находим строки, где есть "из-за"
            words = st[0].split() #бьем строки на слова
            for i in range(len(words)): #убираем удлиненные фонемы в словах, если это не предлог "из-за":
                if len(re.findall(r'(\b[иИ]з-за\b)',words[w])) == 0:
                    words[i] = re.sub(r'([а-я])-\1(?!=|‖)',r'\1', words[i],flags=re.I) 
            words = ' '.join(words) #соединяем все в строку
            st[0] = words    
        else: 
            #В строках, где нет "из-за", просто убираем удлиненные фонемы без каких-либо проверок
            st[0] = re.sub(r'([а-я])-\1(?!=|‖)',r'\1',st[0],flags=re.I) 
           
    for st in text_spld:
        st[0] = re.sub(r'\b([а-я])-\1=?\s[|‖]\s\1',r'\1', st[0],flags=re.I)  #А-а ‖ а он  ---> А он     #З-з= ‖ заглянул --> заглянул:
        st[0] = re.sub(r'([а-я])([а-я])-\2=\s[|‖]\s\1\2', r'\1\2', st[0], flags = re.I)  #На-а= | На-а --> На  
        st[0] = re.sub(r'([а-я])=\s?‖\s=?\1',r'\1', st[0],flags=re.I) #о= ‖ обеспечил --> обеспечил   
        st[0] = re.sub(r'([а-я])=\s?‖\s=?\1',r'\1', st[0],flags=re.I) #как= ‖ =кая ---> какая
       
    #Убираем остальные транскрипционные обозначения:
    for st in text_spld:
        st[0] = re.sub(r'([ǝ~¿+|~—→↓‖∙’«ə»>=,@\*\(\)<0\[\]¡])|(\{\s*[А-Я]+\S*\})|(НРЗБ)|(#[А-Я]+#)|(\xa0)','',st[0]) 
        #почему конец строки не работает? Видимо, нужно установить специальный флаг для этого
    
    #Отдельно обрабатываем случаи типа ужас4н    
    for st in text_spld:
        st[0] = re.sub(r'(?<=[а-я])\d{1,3}[а-я]\b','', st[0]) 
            
    for st in text_spld:
        st[0] = re.sub(r'[0-9]+','', st[0]) #теперь можем убрать цифры во всех остальных случаях
       
    #Обрабатываем случаи: пошё-ол, рождения-а, Всё-е, Я-а расскажу тебе историю, я-ащерицу, ночью-у...
    for st in text_spld:
        st[0] = re.sub(r'(Ё|ё)-о', r'\1', st[0]) #пошё-ол
        st[0] = re.sub(r'(Я|я)-а', r'\1', st[0]) #рождения-а, Я-а, я-ащерицу
        st[0] = re.sub(r'(Ю|ю)-у', r'\1', st[0]) #ночью-у
        st[0] = re.sub(r'(Ё|ё)-е', r'\1', st[0]) #Всё-е
        st[0] = re.sub(r'(Й|й)-(е)', r'\2', st[0])  #Й-ели  Ой ели мы', 'Sib_13-f.eaf\n'
    #pprint(text_spld) 
   
    #Убираем диакритики:
    for st in text_spld:
        nkfd_form = unicodedata.normalize('NFKD', st[0])
        st[0] = u"".join([c for c in nkfd_form if unicodedata.name(c) is not 'COMBINING ACUTE ACCENT'])     
    #unicodedata.combining(chr) Returns the canonical combining class assigned to the character chr as integer. 
    #Returns 0 if no combining class is defined.Combining acute accents is 230
    #if not X: will execute if x is any kind of zero or empty container, or False.
    
    #Убираем лишние пробелы у строк слева и справа: 
    for st in text_spld:
        st[0] = st[0].strip()
        st[1] = st[1].strip()
        words = st[0].split()
        #print(words)
        for i in range(len(words)): #убираем лишние пробелы между словами
            words[i].strip()
            #print(words[i])
        words = ' '.join(words) #соединяем все в строку
        st[0] = words #перезаписываем значение первых строк в text_spld
    pprint(text_spld) 

[['Жил-был один дяденька.', 'Pic-RUS_01-f_Pr-R.eaf'],
 ['У его жены скоро должно было случиться день рождения.',
  'Pic-RUS_01-f_Pr-R.eaf'],
 ['Был случиться.', 'Pic-RUS_01-f_Pr-R.eaf'],
 ['Вот.', 'Pic-RUS_01-f_Pr-R.eaf'],
 ['Дяденька очень долго мучался', 'Pic-RUS_01-f_Pr-R.eaf'],
 ['не знал как обычно выбрать подарок какой получше.',
  'Pic-RUS_01-f_Pr-R.eaf'],
 ['Он ходил по магазинам', 'Pic-RUS_01-f_Pr-R.eaf'],
 ['выбирал', 'Pic-RUS_01-f_Pr-R.eaf'],
 ['думал', 'Pic-RUS_01-f_Pr-R.eaf'],
 ['что бы ему купить...', 'Pic-RUS_01-f_Pr-R.eaf'],
 ['То он хотел купить сумку', 'Pic-RUS_01-f_Pr-R.eaf'],
 ['то он хотел купить часы', 'Pic-RUS_01-f_Pr-R.eaf'],
 ['то манекен', 'Pic-RUS_01-f_Pr-R.eaf'],
 ['но всё у него не получалось.', 'Pic-RUS_01-f_Pr-R.eaf'],
 ['Выбрать', 'Pic-RUS_01-f_Pr-R.eaf'],
 ['что-нибудь стоящее.', 'Pic-RUS_01-f_Pr-R.eaf'],
 ['В итоге он отчаялся', 'Pic-RUS_01-f_Pr-R.eaf'],
 ['пришёл спросить у своих детей:', 'Pic-RUS_01-f_Pr-R.eaf'],
 ['может быть они дадут какой-то

In [488]:
df = pd.DataFrame(text_spld)
df.columns = ['Text', 'ID']
#df
df.to_csv('the whole_dataset.csv', index=False)
#pd.set_option('display.max_rows', df.shape[0]+1)
#df

,0,1
0,Меня зовут Катя,Sib_01-f.eaf
1,И я спала короче.,Sib_01-f.eaf
2,ээ вот,Sib_01-f.eaf
3,я из Казахстана,Sib_01-f.eaf
4,и поэтому я всё лето провела в Казахстане.,Sib_01-f.eaf
5,Полина,Sib_01-f.eaf
6,молчи!,Sib_01-f.eaf
7,Вот.,Sib_01-f.eaf
8,ээ Когда я приехала,Sib_01-f.eaf
9,мы с друзьями решили сходить в горы,Sib_01-f.eaf
